<a href="https://colab.research.google.com/github/t0r8ern1t/other_stuff/blob/main/alpha_hack.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Baseline

In [399]:
import pandas as pd

from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split

## Загрузка данных

In [514]:
train_df = pd.read_parquet('train.parquet')
test_df = pd.read_parquet('test.parquet')

## Обработка данных

Для базовой модели отбросим отдельные таргеты и будем использовать только total_target.

In [515]:
train_df.drop(["id", "target_1", "target_2"], axis=1, inplace=True)

In [516]:
train_df['ogrn_year'] = train_df['ogrn_year'].fillna(0)
train_df.ogrn_year = train_df.ogrn_year.astype(str).astype(int)
import numpy as np

# train_df['cat_col'] = train_df.city
# train_df.cat_col = train_df.cat_col.astype("category")
# cat_columns = train_df.select_dtypes(['category']).columns
# train_df[cat_columns] = train_df[cat_columns].apply(lambda x: x.cat.codes)
# train_df.cat_col.info()


# import seaborn as sns
# sns.scatterplot(data=train_df, x = 'city', y = 'total_target')

In [517]:
a = []
for i in train_df.columns:
  if train_df.dtypes[i] == 'object':
    a.append(i)
    print(i)
train_df.drop(a, axis=1, inplace=True)

nulls = train_df.isnull().sum()/len(train_df)*100
for i in train_df.columns:
    median = train_df[i].median()
    train_df[i].fillna(median, inplace=True)


channel_code
city
city_type
index_city_code
ogrn_month
branch_code
okved
segment


In [518]:
features = list(train_df)

In [519]:
df_target_1 = train_df.query('total_target == 1')
df_target_0 = train_df.query('total_target == 0')
corr1 = df_target_1[features].corr().to_numpy()
corr0 = df_target_0[features].corr().to_numpy()

In [520]:
def if_corrs(x, y):
    if abs(corr0[x][y])+0.2 < abs(corr1[x][y]):
      return True
    else:
      return False

In [521]:
import seaborn as sns

additional_features = []
flag = 0
for i in range(len(features)):
  for j in range(len(features)):
    if abs(corr1[i][j]) > 0.5 and corr1[i][j] != 1:
      if if_corrs(i, j) and [features[j], features[i]] not in additional_features:
        additional_features.append([features[i], features[j]])
        print(corr1[i][j], corr0[i][j], features[i], features[j])
index = 0
for feat in additional_features:
  train_df[f'f{index}'] = train_df[f'{feat[0]}'] * train_df[f'{feat[1]}']
  index += 1

drop_features = []
for i in range(len(features)):
  flag = False
  for j in range(len(features)):
    if i == j:
      continue
    else:
      if abs(corr1[i][j]) > 0.2:
        flag = True
        break
  if not flag and features[i] != 'total_target':
    drop_features.append(features[i])
    print(features[i])
train_df.drop(drop_features, axis=1, inplace=True)

#train_df.head(10)

0.7070074504348652 0.39758927689968737 balance_amt_max sum_deb_e_oper_1m
0.6813657450329798 0.4565501332915982 balance_amt_max sum_cred_e_oper_1m
0.5940081603914732 0.3635850668094151 balance_amt_max sum_deb_e_oper_3m
0.6352166967540054 0.39842831398943485 cnt_days_deb_e_oper_1m cnt_deb_g_oper_1m
0.7388496028674509 0.04285269680081798 cnt_deb_f_oper_1m cnt_days_deb_f_oper_1m
0.6229241095257148 0.043132097621974674 cnt_deb_f_oper_1m cnt_days_deb_f_oper_3m
0.7230131461493428 0.0417851806738491 cnt_days_deb_f_oper_1m cnt_deb_f_oper_3m
0.960691562839762 0.0661046253798805 cnt_cred_f_oper_1m cnt_days_cred_f_oper_1m
0.6885175049093605 0.06993695407473913 cnt_cred_f_oper_1m cnt_days_cred_f_oper_3m
0.8523351041123826 0.5468113644294802 cnt_deb_g_oper_1m cnt_days_deb_g_oper_1m
0.761862953718361 0.5301290010907952 cnt_deb_g_oper_1m cnt_days_deb_g_oper_3m
0.8006368740831472 0.045036285899430324 cnt_deb_f_oper_3m cnt_days_deb_f_oper_3m
0.8611666917045677 0.6451919654641808 cnt_cred_g_oper_3m cnt_d

In [22]:
train_df.cnt_deb_f_oper_1m

0        -0.018435
1        -0.008488
2        -0.018435
3        -0.018435
4         0.001458
            ...   
299995   -0.018435
299996   -0.018435
299997   -0.001858
299998   -0.018435
299999    0.018035
Name: cnt_deb_f_oper_1m, Length: 300000, dtype: float64

In [ ]:
#df_target_0.corr().style.background_gradient(cmap='coolwarm').set_precision(2)

sns.scatterplot(data=train_df,x="total_target", y="max_end_plan_non_fin_deals")

Преобразуем тип категориальных признаков

In [ ]:
cat_cols = a

In [ ]:
train_df[cat_cols] = train_df[cat_cols].astype("category")

## Разбиение на train, validation

In [522]:
X = train_df.drop("total_target", axis=1)
y = train_df.total_target
x_train, x_val, y_train, y_val = train_test_split(X, y,
                                                  test_size=0.2,
                                                  random_state=42)

## Обучение базовой модели

В качестве базовой модели возьмем LGBMClassifier

In [523]:
model = LGBMClassifier(verbosity=-1, random_state=42)
model.fit(x_train, y_train)

LGBMClassifier(random_state=42, verbosity=-1)

In [524]:
y_pred = model.predict_proba(x_val)[:, 1]
roc_auc_score(y_val, y_pred)

0.8766382071805705

In [99]:
# import numpy as np

# city_series = test_df.city.value_counts()
# test_df.city = test_df['city'].fillna(0)
# test_df.city.apply(lambda x: np.nan if str(x).isdigit() else x)
# test_df.city.describe()
# #test_df.city = test_df.city.astype(str).astype(int)

count     100000
unique      3943
top            0
freq       38253
Name: city, dtype: int64

Качество получилось довольно неплохим, но его еще можно улучшить

## Выгрузка результатов

In [506]:
test_df.drop("id", axis=1, inplace=True)
test_df['ogrn_year'] = test_df['ogrn_year'].fillna(0)
test_df.ogrn_year = test_df.ogrn_year.astype(str).astype(int)

# test_df['cat_col'] = test_df.city
# test_df.cat_col = test_df.cat_col.astype("category")
# cat_columns = test_df.select_dtypes(['category']).columns
# test_df[cat_columns] = test_df[cat_columns].apply(lambda x: x.cat.codes)

a = []
for i in test_df.columns:
  if test_df.dtypes[i] == 'object':
    a.append(i)
test_df.drop(a, axis=1, inplace=True)

nulls = test_df.isnull().sum()/len(test_df)*100
for i in test_df.columns:
    median = test_df[i].median()
    test_df[i].fillna(median, inplace=True)


In [507]:
index = 0
for feat in additional_features:
  test_df[f'f{index}'] = test_df[f'{feat[0]}'] * test_df[f'{feat[1]}']
  index += 1

test_df.drop(drop_features, axis=1, inplace=True)

In [508]:
test_score = model.predict_proba(test_df)[:, 1]

In [509]:
sample_submission_df = pd.read_csv("sample_submission.csv")

In [510]:
sample_submission_df.head()

,id,score
0,300000,0.5
1,300001,0.5
2,300002,0.5
3,300003,0.5
4,300004,0.5


In [511]:
sample_submission_df["score"] = test_score

In [512]:
sample_submission_df.head()

,id,score
0,300000,0.169110
1,300001,0.148075
2,300002,0.006123
3,300003,0.006747
4,300004,0.009404


In [513]:
sample_submission_df.to_csv("my_submission.csv", index=False)